# 笔记

## 概括(AI 生成+自己完善)

本教程聚焦于 PyTorch 中数据预处理的核心组件——变换 (Transforms)。它解释了为何需要对数据进行变换 (原始数据通常不符合直接训练的要求)，并强调了 `torchvision` 数据集普遍包含的 `transform` (用于特征) 和 `target_transform` (用于标签) 两个参数。教程以 FashionMNIST 数据集为例，演示了如何使用 `torchvision.transforms.ToTensor` 将 PIL 格式的图像特征转换为归一化的浮点型张量，以及如何使用 `torchvision.transforms.Lambda` 结合 `torch.Tensor.scatter_` 方法将整数标签转换为独热编码 (one-hot encoded) 的张量，从而使数据达到适合模型训练的状态。

## 关键函数、语法(自己总结+AI 优化)

- `torchvision.transforms.Lambda(lambd)`:
    -   是一个通用的变换工具，允许将任何自定义的 `lambda` 函数作为变换来应用。
    -   参数 `lambd`: 表示 `lambda` 函数，接收一个输入 (例如 PIL Image, Tensor) 并返回转换后的输出。
    -   在本教程中被用于 `target_transform`，负责将整数标签转换为独热编码 (one-hot encoded) 的张量。在本教程中，`lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)` 会先创建一个全零张量，随后后在标签 `y` 对应的索引位置上置 `1`。

- `torch.Tensor.scatter_(dim, index, src, reduce=None)` 或 `torch.Tensor.scatter_(dim, index, value, reduce=None)`:
    -   一个**原地 (in-place)** 操作，用于将源张量 (`src`) 或标量值 (`value`) 中的元素按照指定的索引 (`index`) 分散写入到调用该方法的张量 (即 `self`) 中。
    -   `dim`: 进行分散操作的维度。
    -   `index`: 一个张量，包含了要写入数据的索引。`index` 张量的形状可以与 `self` 不同，但其维度数量必须与 `self` 相同，并且在非 `dim` 维度上的大小必须与 `self` 匹配，或者为1（此时会进行广播）。
    -   `src`: 要分散的源张量。其数据类型必须与 `self` 匹配。
    -   `value`: 如果不是分散一个张量 `src`，而是分散一个标量值，则使用此参数。
    -   `reduce` (可选): 指定用于聚合写入到同一索引的多个值的操作，例如 `'add'` 或 `'multiply'`。
    -   在本教程的 `Lambda` 变换中，`scatter_(0, torch.tensor(y), value=1)` 用于在一个大小为10的全零张量中，将由标签 `y` 指定的索引位置（沿着维度0）的值设置为 `1`，从而实现独热编码。

## 提问与解答(自己提问+AI 回答)

- **问题1 (关于对 `ToTensor()` 的描述):**
  教程中提到 `ToTensor()` “converts a PIL image or NumPy `ndarray` into a `FloatTensor`. and scales the image's pixel intensity values in the range [0., 1.]”。
    1.  对于 PIL Image 和 NumPy `ndarray`，`ToTensor()` 期望的输入形状和通道顺序 (例如 HxWxC, CxHxW, RGB, BGR) 是什么，从而能够正确转换为 PyTorch 期望的 CxHxW (通道优先) 张量？
    2.  如果输入的 NumPy 数组已经是 `float` 类型且数值在 `[0., 1.]` 范围内，`ToTensor()` 是否仍会进行缩放？
    3.  `ToTensor()` 除了进行类型转换和值缩放外，是否还进行了其他的操作，例如维度自动重排？

  - **解答:**
    1.  **期望的输入形状和通道顺序:**
        -   **PIL Image**: `ToTensor()` 可以处理多种常见的 PIL 图像模式。
            -   对于 'L' (灰度图，单通道)，输入形状为 (H, W)，输出张量形状为 (1, H, W)。
            -   对于 'RGB' (3通道彩色图)，输入形状为 (H, W, 3)，输出张量形状为 (3, H, W)。
            -   对于 'RGBA' (4通道带alpha的彩色图)，输入形状为 (H, W, 4)，输出张量形状为 (4, H, W)。
            它会将图像数据从 HxWxC 的布局转换为 PyTorch 张量期望的 CxHxW 布局。
        -   **NumPy `ndarray`**: 期望的输入形状是 HxWxC (高度 x 宽度 x 通道数)。例如，一个 RGB 图像应为 `(H, W, 3)`。`ToTensor()` 会将其转换为 CxHxW 的张量。如果输入是单通道灰度图，形状可以是 (H, W)，`ToTensor()` 会将其转换为 (1, H, W)。
        -   `ToTensor()` 本身不进行颜色通道的重新排序 (如 RGB到BGR的转换)。它假设输入的通道顺序是标准的 (例如，PIL 的 'RGB' 模式就是 R, G, B)。

    2.  **对 NumPy 数组类型和缩放行为:**
        -   `ToTensor()` 会将输入的 NumPy `ndarray` 转换为 `torch.FloatTensor`。
        -   关于缩放：其主要行为是将像素值从 `[0, 255]` (通常是 `uint8` 类型图像) 缩放到 `[0.0, 1.0]`。如果输入的 NumPy 数组已经是 `float` 类型，`ToTensor()` 的行为是：它**仍然会假设原始数据范围可能是 `[0, 255]` 并进行除以 `255.0` 的操作**，除非该 `ndarray` 是通过 `torch.from_numpy` 先转换，然后通过某些操作确保其在 `[0,1]` 范围并希望避免再次缩放。最安全的方式是确保输入 `ToTensor` 的 NumPy 数组如果是 `uint8` 类型，则其范围为 `[0, 255]`；如果是 `float` 类型并希望 `ToTensor` 正确处理，最好也先将其调整到 `[0, 255]` 范围或确保你的自定义浮点数范围与 `ToTensor` 的预期行为一致（即它会被除以255）。官方文档通常强调从 `[0, 255]` 到 `[0.0, 1.0]` 的转换，所以对于已经是 `[0.0, 1.0]` 范围的浮点型 NumPy 数组，直接使用 `torch.from_numpy(array).permute(2, 0, 1)` (如果需要维度重排) 可能更直接，或者确保 `ToTensor` 不会错误地再次缩放它（通常它会）。

    3.  **其他操作 (维度重排):**
        -   是的，`ToTensor()` 的一个核心操作就是**维度重排**。无论是 PIL Image 还是 NumPy `ndarray`，如果它们的格式是 HxWxC（高度 x 宽度 x 通道数），`ToTensor()` 会将它们转换为 CxHxW（通道数 x 高度 x 宽度）的张量格式，这是 PyTorch 卷积层等模块期望的输入格式。对于单通道灰度图 (H, W)，它会添加一个通道维度变为 (1, H, W)。

    参考资料:
    -   PyTorch `torchvision.transforms.ToTensor` 官方文档: [https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.ToTensor](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.ToTensor)

- **问题2 (关于使用 `Lambda` 和 `scatter_` 实现独热编码):**
  教程中使用 `Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))` 对标签进行独热编码。
    1.  除了使用 `scatter_`，PyTorch 中是否还有其他函数来实现整数标签转换到独热编码张量？
    2.  `scatter_` 是一个原地操作。在本教程的 `Lambda` 变换中，对新创建的 `torch.zeros(10, ...)` 张量使用原地操作，是否会像在其他 Autograd 场景中那样带来潜在的梯度计算问题？为什么？
    3.  如果分类任务的类别数量非常大 (例如成千上万类)，并且最终使用了像 `nn.CrossEntropyLoss` 这样的损失函数，那么将标签转换为独热编码还是否合适？

  - **解答:**
    1.  **其他独热编码方法:**
        是的，PyTorch 提供了更直接的方法来实现独热编码：
        -   **`torch.nn.functional.one_hot(tensor, num_classes=-1)`**: 这是最推荐和最直接的方法。它接收一个包含类别索引的整数张量，并返回一个独热编码的张量。`num_classes` 参数指定了总类别数。
            ```python
            # 示例:
            # label = torch.tensor(y) # y 是一个整数标签
            # one_hot_label = torch.nn.functional.one_hot(label, num_classes=10)
            ```
        -   **使用单位矩阵索引**: 通过创建一个单位矩阵 `torch.eye(num_classes)`，然后用标签张量作为索引，也可以实现独热编码。
            ```python
            # 示例:
            # label = torch.tensor(y)
            # one_hot_label = torch.eye(10)[label] # 假设有10个类别
            ```
        相比之下，`scatter_` 方法虽然能实现功能，但不如 `torch.nn.functional.one_hot` 来得简洁和专门化。

    2.  **原地操作 `scatter_` 在此场景的影响:**
        在本教程的 `Lambda` 变换中，`scatter_` 被应用于 `torch.zeros(10, dtype=torch.float)` 这个**在 lambda 函数内部即时创建的张量**。
        -   **不会有梯度计算问题**: 因为这个全零张量是在每次调用变换时新生成的，它不是之前某个需要梯度计算的操作的输出，也没有在计算图上积累历史。原地修改这个临时张量不会影响到任何先前存在的、需要梯度的张量的历史记录。Autograd 主要关心的是那些 `requires_grad=True` 并且其值可能被后续梯度计算所依赖的张量被原地修改。在这里，`target_transform` 的输出（即独热编码的张量）才是后续可能被使用的，而它的创建过程虽然包含原地操作，但该操作的对象是局部的、新创建的。

    3.  **独热编码与 `nn.CrossEntropyLoss` 在大规模分类中的适用性:**
        -   **`nn.CrossEntropyLoss` 的期望输入**: PyTorch 的 `nn.CrossEntropyLoss` 损失函数在其内部已经集成了 `LogSoftmax` 和 `NLLLoss`。它期望的模型输出是原始的 logits (未经 softmax 的分数，形状通常为 `[batch_size, num_classes]`)，期望的标签输入是**整数类别的索引** (形状通常为 `[batch_size]`，数据类型为 `torch.long`)。
        -   **独热编码非必需且低效**: 因此，如果使用 `nn.CrossEntropyLoss`，将标签转换为独热编码是**不必要**的，并且当类别数量非常大时会非常**低效**。这会产生一个非常大且稀疏的标签张量，占用大量内存，并可能减慢计算速度。
        -   **推荐做法**: 对于 `nn.CrossEntropyLoss`，直接使用整数形式的类别标签是最优选择。
        -   **何时需要独热编码**: 只有当损失函数明确要求目标是概率分布或类似独热编码的格式时 (例如，如果你自己实现均方误差损失来比较 softmax 输出和独热目标)，才需要进行显式的独热编码。

    参考资料:
    -   `torch.nn.functional.one_hot` 文档: [https://pytorch.org/docs/stable/generated/torch.nn.functional.one_hot.html](https://pytorch.org/docs/stable/generated/torch.nn.functional.one_hot.html)
    -   `nn.CrossEntropyLoss` 文档 (关于期望输入格式): [https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)


[Learn the Basics](intro.html) \|\|
[Quickstart](quickstart_tutorial.html) \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\| **Transforms** \|\| [Build
Model](buildmodel_tutorial.html) \|\|
[Autograd](autogradqs_tutorial.html) \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Transforms
==========

Data does not always come in its final processed form that is required
for training machine learning algorithms. We use **transforms** to
perform some manipulation of the data and make it suitable for training.

All TorchVision datasets have two parameters -`transform` to modify the
features and `target_transform` to modify the labels - that accept
callables containing the transformation logic. The
[torchvision.transforms](https://pytorch.org/vision/stable/transforms.html)
module offers several commonly-used transforms out of the box.

The FashionMNIST features are in PIL Image format, and the labels are
integers. For training, we need the features as normalized tensors, and
the labels as one-hot encoded tensors. To make these transformations, we
use `ToTensor` and `Lambda`.


In [1]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

ds = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

ToTensor()
==========

[ToTensor](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.ToTensor)
converts a PIL image or NumPy `ndarray` into a `FloatTensor`. and scales
the image\'s pixel intensity values in the range \[0., 1.\]


Lambda Transforms
=================

Lambda transforms apply any user-defined lambda function. Here, we
define a function to turn the integer into a one-hot encoded tensor. It
first creates a zero tensor of size 10 (the number of labels in our
dataset) and calls
[scatter\_](https://pytorch.org/docs/stable/generated/torch.Tensor.scatter_.html)
which assigns a `value=1` on the index as given by the label `y`.


In [2]:
target_transform = Lambda(lambda y: torch.zeros(
    10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))

------------------------------------------------------------------------


Further Reading
===============

-   [torchvision.transforms
    API](https://pytorch.org/vision/stable/transforms.html)
